In [ ]:
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain_unstructured import UnstructuredLoader
from langchain_community.vectorstores.utils import filter_complex_metadata



# 0. 임베딩 결과를 저장할 폴더를 지정
cache_dir = LocalFileStore("./.cache/")



# 1. 모델 설정(불러옴)
chat  = ChatOpenAI(model_name="gpt-4o-mini",
                temperature=0.1,
                streaming=True,
                )

# 2. 텍스트 분할기 : 문서를 작은 단위로 쪼갬
# 분할하는 이유는 모델의 토큰 수 제한 및 문서의 길이 때문
# 문서 전체를 맥락으로 제공할 수 없음
# RecursiveCharacterTextSplitter : 의미 있는 단위로 반복적으로 분할 
# 즉 의미있는 문자 구분자 기반으로 재귀 분할
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50, #문장이나 문단을 분할할 때 앞 조각 일부분을 갖고오게 함(맥락 보존)
)

# 2. 텍스트 분할기
# from_tiktoken_encoder : OpenAI의 tiktoken 토크나이저를 사용해 토큰 기준으로 쪼갬
# 줄 바꿈 기준 등 특정 기준으로 쪼개고 토큰 길이로 분할(2단계로 쪼갬)
characterTextSplitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=600,
    chunk_overlap=100,
    separator= "\n", #특정 문자를 찾아 분할함.(분절선이 있는 경우 유용함)
    #langth_function=len, #텍스트를 세는 함수를 지정함.
) # \n 기준으로 나눈 다음, 토큰 수 기준으로 조각화.

# loader =TextLoader("./files/chapter_one.txt")
# PDFloader = PyPDFLoader("./files/chapter_one.pdf")
# 다양한 형식 모두 로드 가능

# 3. 문서 불러오기 및 분할
# 임베딩 모델(OpenAI 등)은 토큰 수 제한이 있고,
# 검색 시, 관련 부분만 선택적으로 제공해야 하기 때문에
# 분할하는 과정은 필수임. 
# loader = UnstructuredFileLoader("./files/chapter_one.pdf")
loader = UnstructuredLoader("./files/chapter_one.pdf")
docs = loader.load_and_split(text_splitter=splitter)
# loader.load()
# PDFloader.load()
# UnstructuredFileLoader.load()

# 3-1 복잡한 메타데이터 필터링
# 문서의 메타데이터를 필터링하여 복잡한 메타데이터를 제거하는 함수
docs = filter_complex_metadata(docs)



# 4. 벡터로 변환 및 캐싱(자연어->숫자 벡터로 임베딩)
embeddings = OpenAIEmbeddings()

# 임베딩 객체와 캐시 디렉토리를 사용하여 CacheBackedEmbeddings 객체를 생성
# CacheBackedEmbeddings는 OpenAIEmbeddings를 래핑하여 캐시된 임베딩을 사용하여 성능을 향상시킴
# cached_embeddings의 역할은 주어진 문장을 해시로 변환하고,
# ./cache에 해당 해시 파일이 있으면 재사용 없으면 API를 호출하여 임베딩을 생성하고 캐시에 저장하는 것
# 즉, 이걸로 감싸기만 하면 자동으로 캐싱이 적용된 임베딩 객체가 됨
# 비싼 OpenAI API 호출을 줄이기 위해 캐시를 사용하여 성능을 향상시킴(속도 증가, 비용 절감);
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# 임베딩을 진행하고, 백터 스토어에 임베딩한 것을 저장
# docs의 텍스트를 cached_embeddings를 사용하여 벡터로 변환하고,
# Chroma 벡터 스토어에 저장함.
# Chroma는 벡터 스토어의 일종으로, 문서와 임베딩을 저장하고 검색하는 데 사용됨
vectorstore = Chroma.from_documents(docs, cached_embeddings)




INFO: pikepdf C++ to Python logger bridge initialized
INFO: Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [2]:
#백터 공간을 검색함(질문에 관련 있는 문서를 찾아 results에 저장)
results = vectorstore.similarity_search("where does winston live")
print(results[0].page_content) #가장 관련 있는 문서의 내용을 출력함

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


row as Winston, a couple of places away. A small, sandy-haired woman who worked in


In [4]:

# 4. 벡터로 변환(자연어->숫자 벡터로 임베딩)
embedder = OpenAIEmbeddings() # 임베딩 인스턴스 생성

vector = embedder.embed_documents([
    "Hi",
    "how",
    "are",
    "you longer sentence"
]) # 4개 문장을 벡터로 변환

print(vector) # 벡터로 변환된 결과
print(len(vector), len(vector[0]))
# 임베딩된 문장 수(4)
# 한 문장이 몇 차원의 벡터로 변환됐는지 보여줌 즉 1536차원

INFO: HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


[[-0.03629858192333018, -0.007224538187570188, -0.03371885554109727, -0.02866363267807191, -0.026865641732513695, 0.03460482274185763, -0.012318847263635718, -0.007752209747023993, 0.0019380524367559983, -0.002701873068082294, 0.02478101390138119, -0.002477124199887517, -0.005732726535614382, -0.002905449946508664, 0.006677323288765644, -0.003032482117949758, 0.03384914384922044, -0.0015032120884641703, 0.021093827586875228, -0.008996472123429598, -0.021719216308744023, 0.01038405247696104, 0.006244111590891486, 0.00708122021044435, -0.012312332661965037, 0.0008998100308185962, 0.005876044512740219, -0.009888952994538026, -0.0030731974470689016, -0.02457255037320985, 0.01074234811826759, -0.013810659381252829, -0.02442923286174532, -0.014110324538845866, 0.0024347802203507035, -0.018878911447619554, 0.0005618723451099323, -0.011270018746398786, 0.018110203351641003, -0.009967126351940971, 0.013028923944578141, -0.011328649230112302, -0.00913327596454606, -0.009654432922329186, -0.02653